# Problem 1 - Working with RDDs (5 points)

This is an interactive PySpark session. Remember that when you open this notebook the `SparkContext` and `SparkSession` are already created, and they are in the `sc` and `spark` variables, respectively. You can run the following two cells to make sure that the Kernel is active.

**Do not insert any additional cells than the ones that are provided.**

In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
spark

In the following cell, make an RDD called `top1m` that contains the contents of the file `top-1m.csv` that you placed into the cluster's HDFS.

In [123]:
top1m=sc.textFile("top-1m.csv")

There is one element in the RDD for each line in the file. The `.count()` method will compute how many lines are in the file. In the following cell, type the expression to count the lines in the `top1m` RDD. Run the cell and see the result.

In [11]:
top1m.count()

1000000

## Count the `.com` domains

How many of the websites in this RDD are in the .com domain?

In the following cell, write a code snippet that finds the records with `.com` and counts them. (Hint: use a regular expression.)

In [43]:
def hascom( s ):
    return ".com" in s

In [44]:
a = top1m.filter(hascom)

In [45]:
a.count()

537633

In [48]:
a.take(20)

[u'1,google.com',
 u'2,youtube.com',
 u'3,facebook.com',
 u'4,baidu.com',
 u'6,yahoo.com',
 u'7,qq.com',
 u'8,amazon.com',
 u'9,taobao.com',
 u'10,twitter.com',
 u'12,tmall.com',
 u'13,instagram.com',
 u'14,live.com',
 u'15,vk.com',
 u'16,sohu.com',
 u'17,jd.com',
 u'18,sina.com.cn',
 u'19,reddit.com',
 u'20,weibo.com',
 u'24,blogspot.com',
 u'25,login.tmall.com']

## Histogram the Top Level Domains (TLDs)

What is the distribution of TLDs in the top 1 million websites? We can quickly compute this using the RDD function `countByValue()`.

In the following cell, write a function called `tld` (in Python) that takes a domain name string and outputs the top-level domain.

In [148]:
import re
def tld(line):
    line=re.sub(r'\d+,','', line)
    return "{}".format(line)


In the following cell, map the `top1m` RDD using `tld` into a new RDD called `tlds`. 

In [171]:
tlds=top1m.map( lambda line:tld( line ))

In the following two cells, evaluate `top1m.first()` and  `tlds.first()` to see if the first line of `top1m` transformed by `tld` is properly represented as the first line of `tlds`. 

In [ ]:
top1m.first()

In [172]:
tlds.first()

'google.com'

Look at the first 50 elements of `top1m` by evaluating `top1m.take(50)`.

In [158]:
top1m.take(50)

[u'1,google.com',
 u'2,youtube.com',
 u'3,facebook.com',
 u'4,baidu.com',
 u'5,wikipedia.org',
 u'6,yahoo.com',
 u'7,qq.com',
 u'8,amazon.com',
 u'9,taobao.com',
 u'10,twitter.com',
 u'11,google.co.in',
 u'12,tmall.com',
 u'13,instagram.com',
 u'14,live.com',
 u'15,vk.com',
 u'16,sohu.com',
 u'17,jd.com',
 u'18,sina.com.cn',
 u'19,reddit.com',
 u'20,weibo.com',
 u'21,google.co.jp',
 u'22,yandex.ru',
 u'23,360.cn',
 u'24,blogspot.com',
 u'25,login.tmall.com',
 u'26,linkedin.com',
 u'27,pornhub.com',
 u'28,google.ru',
 u'29,netflix.com',
 u'30,google.com.br',
 u'31,google.com.hk',
 u'32,google.co.uk',
 u'33,bongacams.com',
 u'34,yahoo.co.jp',
 u'35,google.fr',
 u'36,csdn.net',
 u'37,t.co',
 u'38,google.de',
 u'39,ebay.com',
 u'40,microsoft.com',
 u'41,alipay.com',
 u'42,office.com',
 u'43,twitch.tv',
 u'44,msn.com',
 u'45,bing.com',
 u'46,xvideos.com',
 u'47,microsoftonline.com',
 u'48,mail.ru',
 u'49,pages.tmall.com',
 u'50,ok.ru']

Try the same thing with the `tlds` RDD to make sure that the first 50 lines were properly transformed.


In [173]:
tlds.take(50)

['google.com',
 'youtube.com',
 'facebook.com',
 'baidu.com',
 'wikipedia.org',
 'yahoo.com',
 'qq.com',
 'amazon.com',
 'taobao.com',
 'twitter.com',
 'google.co.in',
 'tmall.com',
 'instagram.com',
 'live.com',
 'vk.com',
 'sohu.com',
 'jd.com',
 'sina.com.cn',
 'reddit.com',
 'weibo.com',
 'google.co.jp',
 'yandex.ru',
 '360.cn',
 'blogspot.com',
 'login.tmall.com',
 'linkedin.com',
 'pornhub.com',
 'google.ru',
 'netflix.com',
 'google.com.br',
 'google.com.hk',
 'google.co.uk',
 'bongacams.com',
 'yahoo.co.jp',
 'google.fr',
 'csdn.net',
 't.co',
 'google.de',
 'ebay.com',
 'microsoft.com',
 'alipay.com',
 'office.com',
 'twitch.tv',
 'msn.com',
 'bing.com',
 'xvideos.com',
 'microsoftonline.com',
 'mail.ru',
 'pages.tmall.com',
 'ok.ru']

At this point, `tlds.countByValue()` would give us a list of each TLD and the number of times that it appears in the top1m file. Note that this function returns the results as a `defaultDict` in the Python environemnt, not as an RDD. But we want it reverse sorted by count. To do this, we can set a variable called `tlds_and_counts` equal to `tlds.countByValue()` and then reverse the order, sort, and take the top 50, like this:

```
tlds_and_counts = tlds.countByValue()
counts_and_tlds = [(count,domain) for (domain,count) in tlds_and_counts.items()]
```

In the following cell, run the code above to produce the Python Dictionary.

In [174]:
tlds_and_counts = tlds.countByValue()
counts_and_tlds = [(count,domain) for (domain,count) in tlds_and_counts.items()]

In the following cell, reverse sort `counts_and_tlds` and display the first 50.

In [175]:
counts_and_tlds.

[(1, 'dutildenim.com'),
 (1, 'buffdaddy.com'),
 (1, 'elektroinzynieria.pl'),
 (1, 'enzolifesciences.com'),
 (1, 'gategoldfitnessproducts.com'),
 (1, 'thegreenhousedriftwood.com'),
 (1, 'netadclick.com'),
 (1, 'fipi.ru'),
 (1, 'pokochkam.ru'),
 (1, 'homosexualpanicattack.tumblr.com'),
 (1, 'themecloset.com'),
 (1, 'ejemplosdecv.com'),
 (1, '4-fotos-1-palabra.blogspot.com'),
 (1, 'addisongallery.com'),
 (1, 'thextremexperience.com'),
 (1, 'oraesatta.info'),
 (1, 'tui.fi'),
 (1, 'jogtar.hu'),
 (1, 'westsquareboston.com'),
 (1, 'surverynerds.com'),
 (1, 'waimeavalley.net'),
 (1, 'arcadespain.info'),
 (1, 'gaonsitelabs.com'),
 (1, 'bizzdesign.com'),
 (1, 'tkc.co.jp'),
 (1, 'fuel-economy.co.uk'),
 (1, 'mogliescopata.com'),
 (1, 'tui.fr'),
 (1, 'mecsoft.com'),
 (1, 'teachbase.ru'),
 (1, 'hronokuhinja.rs'),
 (1, 'bostonseed.com'),
 (1, 'earnwithdrop.com'),
 (1, 'hotsale.com.ar'),
 (1, 'forum-aktiv.com'),
 (1, 'wordart.com'),
 (1, 'acupunctureandbeautyclinic.com'),
 (1, 'ez4usms.com'),
 (1, 'to

**Question:** `top1m.collect()[0:50]` and `top1m.take(50)` produce the same result. Which one is more efficient and why? Put your answer in the cell below.

In [ ]:
## Answer Here (don't run this cell)

When you finish this problem, click on the File -> 'Save and Checkpoint' in the menu bar to make sure that the latest version of the workbook file is saved. Also, before you close this notebook and move on, make sure you disconnect your SparkContext, otherwise you will not be able to re-allocate resources. Remember, you will commit the .ipynb file to the repository for submission (in the master node terminal.)

In [ ]:
sc.stop()